In [ ]:
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import matplotlib.pyplot as plt # data visualization library
from sklearn import datasets
from sklearn.datasets import fetch_olivetti_faces
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier

# Exercise: k-means Initialization

We use for the evaluation of the correct answer seed fixing for random generators in sklearn and numpy. Hence, make sure that you have the newest versions (sklearn version 1.2.0 and numpy version 1.24.1).

In [ ]:
!pip3 show scikit-learn

In [ ]:
!pip3 show numpy

If your versions don't match, the following commands (or their anaconda version) could help to get the newest stable release. If you need help with this, please ask the TAs during instruction hours.

In [ ]:
!pip3 install scikit-learn --upgrade
!pip3 install numpy --upgrade

The functions generating the datasets are given here:

In [ ]:
def generateMoons(epsilon, n):
    moons, labels = datasets.make_moons(n_samples=n, noise=epsilon, random_state=7)
    return moons, labels, 2
def generateBlobs(epsilon, n):
    blobs, labels = datasets.make_blobs(n_samples=n,centers=3, cluster_std=[epsilon + 1, epsilon + 1.5, epsilon + 0.5], random_state=54)
    return blobs, labels, 3
def generateAniso(epsilon,n):
    # Anisotropicly distributed data
    A, labels = datasets.make_blobs(n_samples=n, random_state=170)
    transformation = [[0.6, -0.6], [-0.4, 0.8]]
    aniso = np.dot(A, transformation)
    return aniso, labels, 3

Implement the centroid initialization here. Right now, it returns a random sample from the datapoints. 

In [ ]:
def init_centroids_greedy_pp(D,r,l=10):
    '''
        :param r: (int) number of centroids (clusters)
        :param D: (np-array) the data matrix
        :param l: (int) number of centroid candidates in each step
        :return: (np-array) 'X' the selected centroids from the dataset
    '''   
    #Do not change the seed
    rng =  np.random.default_rng(seed=7) # use this random generator to sample the candidates (sampling according to given probabilities can be done via rng.choice(..))
    n,d = D.shape

    indexes = rng.integers(low=0, high=n, size=r)
    X = np.array(D[indexes,:]).T
    return X

This is the $k$-means implementation from the lecture accompanying notebook.

In [ ]:
def RSS(D,X,Y):
    return np.sum((D- Y@X.T)**2)

In [ ]:
def getY(labels):
    '''
        Compute the cluster assignment matrix Y from the categorically encoded labels
    '''
    Y = np.eye(max(labels)+1)[labels]
    return Y
def update_centroid(D,Y):
    cluster_sizes = np.diag(Y.T@Y).copy()
    cluster_sizes[cluster_sizes==0]=1
    return D.T@Y/cluster_sizes
def update_assignment(D,X):
    dist = np.sum((np.expand_dims(D,2) - X)**2,1)
    labels = np.argmin(dist,1)
    return getY(labels)
def kmeans(D,r, X_init, epsilon=0.00001, t_max=10000):
    X = X_init.copy()
    Y = update_assignment(D,X)
    rss_old = RSS(D,X,Y) +2*epsilon
    t=0
    #Looping as long as difference of objective function values is larger than epsilon
    while rss_old - RSS(D,X,Y) > epsilon and t < t_max-1:
        rss_old = RSS(D,X,Y)
        X = update_centroid(D,Y)
        Y = update_assignment(D,X)
        t+=1
    print(t,"iterations")
    return X,Y

We generate the datasets.

In [ ]:
n=500
D_blobs, labels_blobs, r_blobs = generateBlobs(0.05,n)
D_moons, labels_moons, r_moons = generateMoons(0.05,n)
D_aniso, labels_aniso, r_aniso = generateAniso(0.05,n)

In [ ]:
D, labels, r = D_blobs, labels_blobs, r_blobs
X_init = init_centroids_greedy_pp(D,r)
X,Y = kmeans(D,r, X_init)

Plot the clustering. The initial centroids are marked in red, and the final centroids are marked in blue.

In [ ]:
fig = plt.figure()
plt.scatter(D[:, 0], D[:, 1], c=np.argmax(Y,axis=1), s=10)
plt.scatter(X_init.T[:, 0], X_init.T[:, 1], c='red', s=50, marker = 'D')
plt.scatter(X.T[:, 0], X.T[:, 1], c='blue', s=50, marker = 'D')

In [ ]:
from sklearn.metrics import normalized_mutual_info_score

D_m, labels_m, r_m = D_moons, labels_moons, r_moons
X_init_m = init_centroids_greedy_pp(D_m,r_m)
X_m,Y_m = kmeans(D_m,r_m, X_init_m)

D_a, labels_a, r_a = D_aniso, labels_aniso, r_aniso
X_init_a = init_centroids_greedy_pp(D_a,r_a)
X_a,Y_a = kmeans(D_a,r_a, X_init_a)

predicted_labels = Y.argmax(axis=1)
nmi_score = normalized_mutual_info_score(labels, predicted_labels)
nmi_score_rounded = round(nmi_score, 2)

print("NMI score(Blob):", nmi_score_rounded)

predicted_labels_m = Y_m.argmax(axis=1)
nmi_score_m = normalized_mutual_info_score(labels_m, predicted_labels_m)
nmi_score_rounded_m = round(nmi_score_m, 2)

predicted_labels_a = Y_a.argmax(axis=1)
nmi_score_a = normalized_mutual_info_score(labels_a, predicted_labels_a)
nmi_score_rounded_a = round(nmi_score_a, 2)

print("NMI score(Aniso):", nmi_score_rounded_a)
print("NMI score(Moon):", nmi_score_rounded_m)

# Q1a)A-Correct, B-Correct, C-Correct, D-Wrong
# Q1b)0.95, 0.19, 0.59

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import numpy as np
import math



# Q2
#A-Correct
data = {'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'X': [13.40, 6.29, 10.68,12.04, 6.10, 0.77,7.64, 2.26, -0.31,0.29],
        'Y': ['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No']}
yes_data = [data['X'][i] for i in range(len(data['X'])) if data['Y'][i] == 'Yes']
class_specific_mean = sum(yes_data) / len(yes_data)
print("The class-specific mean of X of the companies that are 'Yes':", class_specific_mean)



#B-Incorrect

# Assume we have data on past companies, where the first column is the percent profit from the previous year and the second column is whether or not the company issued a dividend
# 0 = No, 1 = Yes
data = np.array([[13.40,1], [6.29,1], [10.68,1], [12.04,1], [6.10,1], [0.77,1], [7.64,0], [2.26,0], [-0.31,0], [0.29,0]])

# Split the data into the feature (X) and target (y) variables
X = data[:,0]
y = data[:,1]

# Create an instance of the Gaussian Naive Bayes model
gnb = GaussianNB()

# Train the model on the data
gnb.fit(X.reshape(-1,1), y)

# Make a prediction on a new data point (unit)
X_new = np.array([4])
pred_proba = gnb.predict_proba(X_new.reshape(-1,1))

# calculate the mean and standard deviation of X
mean = np.mean(X)
sigma = math.sqrt(np.sum((x - mean) ** 2 for x in X) / (len(X) - 1))

# calculate the posterior odds ratio
posterior_odds_ratio = pred_proba[0][0] / pred_proba[0][1]
print("The posterior odds ratio for a company who hypothetically had last year's profit X = 4 is {:.4f}".format(posterior_odds_ratio))



#C-Correct

data = np.array([[13.40,1], [6.29,1], [10.68,1], [12.04,1], [6.10,1], [0.77,1], [7.64,0], [2.26,0], [-0.31,0], [0.29,0]])

# Split the data into the feature (X) and target (y) variables
X = data[:,0].reshape(-1,1)
y = data[:,1]

# Create an instance of the Gaussian Naive Bayes model
gnb = GaussianNB()

# Train the model on the data
gnb.fit(X, y)

# Make a prediction on a new data point (unit)
X_new = np.array([0.77]).reshape(-1,1)
pred = gnb.predict(X_new)

if pred == 0:
    print("According to the Naive Bayes model, the sixth unit (ID=6) is classified in the negative class (Class 'No').")
else:
    print("According to the Naive Bayes model, the sixth unit (ID=6) is classified in the positive class (Class 'Yes').")

    
#D-Incorrect  
    
data = np.array([[13.40,1], [6.29,1], [10.68,1], [12.04,1], [6.10,1], [0.77,1], [7.64,0], [2.26,0], [-0.31,0], [0.29,0]])

# Split the data into the feature (X) and target (y) variables
X = data[:,0].reshape(-1,1)
y = data[:,1]

# Create an instance of the Gaussian Naive Bayes model
gnb = GaussianNB()

# Train the model on the data
gnb.fit(X, y)

# Make predictions on the training data
y_pred = gnb.predict(X)

# Calculate the accuracy
accuracy = accuracy_score(y, y_pred)

print("The accuracy of the Naive Bayes model on the training data is {:.2f}%".format(accuracy*100))

# Exercise: Decision tree

In [ ]:
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split

Load the data

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

Splitting the data - 80:20 ratio

Use also for this exercise the latest stable version of sklearn because of the use of the random generator for the train and test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size = 0.2, random_state = 42)

tree = DecisionTreeClassifier()
fitted_tree = tree.fit(X_train, y_train)
y_pred = fitted_tree.predict(X_test)

print("Training split input- ", X_train.shape)
print("Testing split input- ", X_test.shape)

Function to visualize the decision tree (the argument is a fitted sklearn DecisionTreeClassifier)

In [ ]:
def plot_iris_tree(decision_tree):
        fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=300)
        plot_tree(decision_tree=decision_tree, feature_names = ["sepal_length","sepal_width","petal_length","petal_width"], 
                     class_names =["setosa", "versicolor", "verginica"] , filled = True , precision = 2, rounded = True)
plot_iris_tree(fitted_tree)

A - True, B - True, C - False, D - False

# Exercise : Stochastic Gradient Descent

Scikit learn provides two classes `SGDRegressor` and `SGDClassifier` which use stochastic gradient descent to carry out linear regression and classification respectively.

These models are especially useful in these situations:
- with very large datasets
- with streaming data (they support online learning)
- with datasets with sparse features

SGD is sensitive to learning rate and the scale of the features. It's advisable to z-score the features, and to tune the learning rate.

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_absolute_error

We'll use the dataset of 50,000 songs. The prediction task is to guess the year the song was made based on 90 timbre features extracted from the audio. The year is in the first column.

In [ ]:
songs = np.load("C:/Users/snipe/OneDrive - TU Eindhoven/UNI Y2/Q2/JBI030/HW3/songs50k.npy")
X_train, X_val, y_train, y_val = train_test_split(songs[:,1:], songs[:,0], test_size=1/3, random_state=10)

Train and evaluate the SGD classifier on the songs dataset. Take the following steps:
- z-score the training and validation features
- find good settings for learning rate type and learning rate initial value.
  - r-squared on validation data 
  - mean absolute error on validation data  

We scale the data using the StandardScaler

In [ ]:
scaler=StandardScaler()
X_train_sc=scaler.fit_transform(X_train)
X_val_sc=scaler.transform(X_val)

Use also for this exercise the latest stable version of sklearn because of the use of the random generator for the train and test split.

Use the SGDRegressor with a random state of 123, that is evaluate the parameter settings like this:

In [ ]:
learning_rate=["optimal", "constant","invscaling"]
eta0= [1, 0.1, 0.01, 0.001, 0.0001]
loss = ["huber", "squared_loss"]
model = SGDRegressor(learning_rate=learning_rate[0],eta0=eta0[0],random_state=123,loss=loss[0])
model.fit(X_train_sc, y_train)

In [ ]:
def  combinationsR2(learning_rate, eta0, loss):
    dictR2 = {}
    for i in learning_rate:
        for k in eta0:
            for l in loss:
                model = SGDRegressor(learning_rate=i,eta0=k,random_state=123,loss=l)
                model.fit(X_train_sc, y_train)
                y_pred = model.predict(X_val_sc)
                dictR2[i,k,l] = r2_score(y_val, y_pred)
    return dictR2

In [ ]:
def  combinationsMSE(learning_rate, eta0, loss):
    dictMSE = {}
    for i in learning_rate:
        for k in eta0:
            for l in loss:
                model = SGDRegressor(learning_rate=i,eta0=k,random_state=123,loss=l)
                model.fit(X_train_sc, y_train)
                y_pred = model.predict(X_val_sc)
                dictMSE[i,k,l] = mean_absolute_error(y_val, y_pred)
    return dictMSE

In [ ]:
dictR2 = combinationsR2(learning_rate, eta0, loss)
dictMAE = combinationsMSE(learning_rate, eta0, loss)

In [ ]:
#A False
bestR2 = max(dictR2.values())
bestR2 == 0.28

In [ ]:
#B False
lowestR2 = dictR2['optimal', 1, 'huber']
lowestR2 == min(dictR2.values())

In [ ]:
#C False
lowestMAE = dictMAE['optimal', 1, 'huber']
lowestMAE == min(dictMAE.values())

In [ ]:
#D True
dictR2['invscaling', 0.01, 'huber'] - dictR2['invscaling', 0.1, 'huber']

By default SGDRegressor tries to minimize the standard linear regression error function, that is sum of squared error. However this can be changed, via the `loss=` parameter. When `loss='squared_loss'`, sum of squared errors will be used. Other error functions available include [Huber loss](https://en.wikipedia.org/wiki/Huber_loss) (`loss='huber'`). Compared to squared loss, huber focuses less on outliers.

# Exercise : Latent Space

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

The method `scatter_pictures` displays a scatter plot where the coordinates are given by `Y` and the data points are visualized by pictures of the faces defined in `D_approx`. You can try using `D_approx=D` as argument or you could insert the approximation of the pictures as computed by the low-rank matrix factorization (SVD) into `D_approx`.

In [ ]:
def scatter_pictures(D_approx, Y):
    zoom = 0.4
    fig, ax = plt.subplots(figsize=(16,16))
    for i in range(D.shape[0]):
        image = D_approx[i,:].reshape((64, 64))
        im = OffsetImage(image, cmap=plt.cm.gray,zoom=zoom)
        ab = AnnotationBbox(im, (Y[i,0], Y[i,1]), xycoords='data', frameon=False)
        ax.add_artist(ab)
        ax.update_datalim([(Y[i,0], Y[i,1])])
        ax.autoscale()

In [ ]:
faces = fetch_olivetti_faces()
D = faces.data

In [ ]:
scatter_pictures(D, np.random.rand(D.shape[0],2))

## 5

In [ ]:
r=2
U, s, V = np.linalg.svd(D,full_matrices=True)
Y=U[:,0:r]*np.sqrt(s[0:r])
X=V.T[:,0:r]*np.sqrt(s[0:r])
D_trunc = Y@X.T
scatter_pictures(D_trunc,Y)

In [ ]:
r=100
U, s, V = np.linalg.svd(D,full_matrices=True)
Y=U[:,0:r]*np.sqrt(s[0:r])
X=V.T[:,0:r]*np.sqrt(s[0:r])
D_trunc = Y@X.T
scatter_pictures(D_trunc,Y)

From the visualization with r=2 we can see that one of the features corresponds to whether light is coming from left or right side of the picture (Y-axis). The other axis can only be distincted when r is much higher - like r=100. Then we can instantly notice that pictures on the right represent people with darker skin tone, less light, or dark facial hair, while on the left we can only see caucasians without any hair in the frame and high exposition of the picture.

- X-axis - Overall amount of light/dark pixels (e.g. skin tone, black hair, picture exposition)
- Y-axis - light on left/right

Latent space (Y) represents feature intensity in specific pictures. First column represents how bright the picture is - the higher the value, the brighter it is. Second column informs how skewed to the left the origin of light is - the higher the value, the more light we see coming from the left side andless from the right side.